In [13]:
import os
import pprint
import shutil
import importlib 

from climatools.cliradlw import (setup, pipeline)
import climatools.cliradlw.runrecord as runrecord_clirad
import climatools.cliradlw.utils as utils_clirad


from climatools.lblnew import setup_overlap
from climatools.lblnew import setup_bestfit
import climatools.lblnew.pipeline as pipeline_lblnew
import climatools.lblnew.bestfit_params as bestfit
import climatools.lblnew.runrecord_bestfit as runrecord_bestfit
import climatools.lblnew.runrecord_overlap as runrecord_overlap



importlib.reload(setup)
importlib.reload(pipeline)
importlib.reload(runrecord_clirad)
importlib.reload(utils_clirad)
importlib.reload(setup_overlap)
importlib.reload(setup_bestfit)
importlib.reload(pipeline_lblnew)
importlib.reload(bestfit)
importlib.reload(runrecord_bestfit)
importlib.reload(runrecord_overlap)

<module 'climatools.lblnew.runrecord_overlap' from '/chia_cluster/home/jackyu/climatools/climatools/lblnew/runrecord_overlap.py'>

In [20]:
def params_clirad_nonoverlap():
    '''
    Define clirad-lw input parameter dictionaries
    for co2 band 4.
    '''
    commitnumber = '913ed72'
    molecule = {'co2': 400e-6}
    bands = [4,] 
    atmpros = [('mls', 294), ('saw', 257), ('trp', 300)]
    
    params = []
    for band in bands:
        for atmpro, tsfc in atmpros:
            param = {'commitnumber': commitnumber,
                     'molecule': molecule,
                     'band': [band],
                     'atmpro': atmpro,
                     'tsfc': tsfc}
            params.append(param)
    return params



def param_lblnewbestfit():
    '''    
    lblnew-bestfit input parameter dictionaries
    matching those from clirad-lw's.
    '''
    commitnumber = 'a22ab94'
    
    params_clirad = params_clirad_nonoverlap()
    record_bestfit = runrecord_bestfit.params()
    
    params = []
    for param_clirad in params_clirad:
        
        molecule, conc = list(param_clirad['molecule'].items())[0]
        band = param_clirad['band'][0]
        band = utils_clirad.mapband_new2old()[band]
        atmpro = param_clirad['atmpro']
        tsfc = param_clirad['tsfc']
        
        param_bestfit = [r for r in record_bestfit
                         if r['commitnumber'] == commitnumber
                         if r['molecule'] == molecule
                         if r['conc'] == conc
                         if r['band'] == band
                         if r['atmpro'] == atmpro
                         if r['tsfc'] == tsfc]
        
        assert len(param_bestfit) == 1
        param_bestfit = param_bestfit[0]
        params.append(param_bestfit)
        
    return params


def parampairs_nonoverlap():
    params_clirad = params_clirad_nonoverlap()
    params_lblnew = param_lblnewbestfit()
    
    assert len(params_clirad) == len(params_lblnew)
    
    return list(zip(params_clirad, params_lblnew))




In [16]:
parampairs_nonoverlap()

[({'atmpro': 'mls',
   'band': [4],
   'commitnumber': '3e8a8c5',
   'molecule': {'co2': 0.0004},
   'tsfc': 294},
  {'atmpro': 'mls',
   'band': '3b',
   'commitnumber': 'a22ab94',
   'conc': 0.0004,
   'dv': 0.001,
   'klin': 0,
   'molecule': 'co2',
   'ng_adju': [0, 0],
   'ng_refs': [5, 2],
   'nv': 1000,
   'option_compute_btable': 0,
   'option_compute_ktable': 0,
   'option_wgt_flux': 1,
   'option_wgt_k': 1,
   'ref_pts': [(1, 250), (10, 250)],
   'tsfc': 294,
   'vmax': 720,
   'vmin': 620,
   'w_diffuse': [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)],
   'wgt': [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]}),
 ({'atmpro': 'saw',
   'band': [4],
   'commitnumber': '3e8a8c5',
   'molecule': {'co2': 0.0004},
   'tsfc': 257},
  {'atmpro': 'saw',
   'band': '3b',
   'commitnumber': 'a22ab94',
   'conc': 0.0004,
   'dv': 0.001,
   'klin': 0,
   'molecule': 'co2',
   'ng_adju': [0, 0],
   'ng_refs': [5, 2],
   'nv': 1000,
   'option_compute_btable': 0,
   'option_compute_ktable': 0,
   'o

In [17]:
def script_clirad_lblnew():
    '''
    Script to run clirad and then plot results against the 
    corresponding lblnew case.  This needs both input parameter
    dictionaries from clirad and lblnew.
    '''
    ps = parampairs_nonoverlap()
    
    gprocs = pipeline.pipeline_fortran2ipynb2git(parampairs=ps, 
                                                 setup=setup)
    


In [18]:
pprint.pprint(list(zip(*parampairs_nonoverlap()))[0])

({'atmpro': 'mls',
  'band': [4],
  'commitnumber': '3e8a8c5',
  'molecule': {'co2': 0.0004},
  'tsfc': 294},
 {'atmpro': 'saw',
  'band': [4],
  'commitnumber': '3e8a8c5',
  'molecule': {'co2': 0.0004},
  'tsfc': 257},
 {'atmpro': 'trp',
  'band': [4],
  'commitnumber': '3e8a8c5',
  'molecule': {'co2': 0.0004},
  'tsfc': 300})


In [19]:
script_clirad_lblnew()

Submitting radiation calculation for cases
{'atmpro': 'mls',
 'band': [4],
 'commitnumber': '3e8a8c5',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [4],
 'commitnumber': '3e8a8c5',
 'molecule': {'co2': 0.0004},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [4],
 'commitnumber': '3e8a8c5',
 'molecule': {'co2': 0.0004},
 'tsfc': 300}

Submitting analysis for cases
{'atmpro': 'mls',
 'band': [4],
 'commitnumber': '3e8a8c5',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [4],
 'commitnumber': '3e8a8c5',
 'molecule': {'co2': 0.0004},
 'tsfc': 257}
{'atmpro': 'trp',
 'band': [4],
 'commitnumber': '3e8a8c5',
 'molecule': {'co2': 0.0004},
 'tsfc': 300}

Committing analysis to Git repository for cases
{'atmpro': 'trp',
 'band': [4],
 'commitnumber': '3e8a8c5',
 'molecule': {'co2': 0.0004},
 'tsfc': 300}
{'atmpro': 'mls',
 'band': [4],
 'commitnumber': '3e8a8c5',
 'molecule': {'co2': 0.0004},
 'tsfc': 294}
{'atmpro': 'saw',
 'band': [4],
 'commitnumbe

In [22]:
for param, _ in parampairs_nonoverlap():
    print(pipeline.nbviewer_url(param=param, setup=setup))
    print()


http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/co2_0.0004/band_4/atmpro_mls/cliradlw_913ed72/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/co2_0.0004/band_4/atmpro_saw/cliradlw_913ed72/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/co2_0.0004/band_4/atmpro_trp/cliradlw_913ed72/results_cliradlw.ipynb




http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/co2_0.0004/band_4/atmpro_mls/cliradlw_3e8a8c5/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/co2_0.0004/band_4/atmpro_saw/cliradlw_3e8a8c5/results_cliradlw.ipynb

http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/clirad/co2_0.0004/band_4/atmpro_trp/cliradlw_3e8a8c5/results_cliradlw.ipynb